In [22]:
!python -m ipykernel install --user --name=myenv

Installed kernelspec myenv in C:\Users\arman\AppData\Roaming\jupyter\kernels\myenv


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
import warnings

warnings.filterwarnings('ignore')

# تنظیمات
SEED = 42
SEQ_LENGTH = 60
BATCH_SIZE = 64
EPOCHS = 200
torch.manual_seed(SEED)
np.random.seed(SEED)

# کلاس Dataset
class PriceDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return (
            torch.FloatTensor(self.sequences[idx]),
            torch.FloatTensor([self.targets[idx]]).squeeze()
        )

# مدل LSTM بهبود یافته
class EnhancedLSTM(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=64,
            num_layers=1,
            batch_first=True,
            dropout=0.1
        )
        self.fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.1),
            nn.Linear(32, 1)
        )
        
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :]).squeeze()

# بارگذاری و پیش‌پردازش داده
def load_and_preprocess(file_path):
    df = pd.read_csv(file_path, sep='\t', header=0)
    df['datetime'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    df = df.drop(['<DATE>', '<TIME>'], axis=1)
    df.columns = [col.strip('<>').lower() for col in df.columns]
    df = df.set_index('datetime').sort_index()
    
    # محاسبه لگاریتم قیمت‌ها
    df['log_close'] = np.log(df['close'])
    
    # ویژگی‌های فنی بر اساس لگاریتم قیمت
    df['log_return'] = df['log_close'].diff()
    df['volatility'] = df['log_return'].rolling(20).std().fillna(0)
    df['price_change'] = df['log_close'].diff(5)
    
    # اندیکاتورهای تکنیکال
    df.ta.ema(length=20, close='log_close', append=True)
    df.ta.rsi(length=14, close='log_close', append=True)
    
    # حذف مقادیر نامعتبر
    df = df.dropna().replace([np.inf, -np.inf], 0)
    
    # هدف پیش‌بینی: تغییرات لگاریتمی قیمت
    df['target'] = df['log_close'].shift(-1) - df['log_close']
    
    return df[['log_close', 'log_return', 'volatility', 'price_change', 
              'EMA_20', 'RSI_14', 'target']]

# آماده‌سازی داده
def prepare_data(df):
    # نرمال‌سازی
    features = df.drop(columns=['target'])
    target = df['target']
    
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    
    # ایجاد دنباله‌ها
    sequences, targets = [], []
    for i in range(len(scaled_features)-SEQ_LENGTH):
        sequences.append(scaled_features[i:i+SEQ_LENGTH])
        targets.append(target.iloc[i+SEQ_LENGTH])
    
    return np.array(sequences), np.array(targets), scaler

# آموزش مدل
def train_model(model, train_loader, test_loader):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    criterion = nn.MSELoss()
    
    best_loss = float('inf')
    early_stop_counter = 0
    history = {
        'train_loss': [],
        'test_loss': [],
        'rmse': [],
        'r2': []
    }
    
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item()
        
        # ارزیابی
        model.eval()
        test_loss, preds, truths = 0, [], []
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                test_loss += criterion(outputs, labels).item()
                preds.extend(outputs.numpy())
                truths.extend(labels.numpy())
        
        # محاسبه معیارها
        avg_train_loss = train_loss / len(train_loader)
        avg_test_loss = test_loss / len(test_loader)
        rmse = np.sqrt(mean_squared_error(truths, preds))
        r2 = r2_score(truths, preds)
        
        # ذخیره تاریخچه
        history['train_loss'].append(avg_train_loss)
        history['test_loss'].append(avg_test_loss)
        history['rmse'].append(rmse)
        history['r2'].append(r2)
        
        # چاپ گزارش
        print(f'Epoch {epoch+1}/{EPOCHS}')
        print(f'Train Loss: {avg_train_loss:.6f} | Test Loss: {avg_test_loss:.6f}')
        print(f'RMSE: {rmse:.6f} | R²: {r2:.6f}')
        print('-' * 60)
        
        # Early Stopping و تنظیم نرخ یادگیری
        scheduler.step(avg_test_loss)
        
        if avg_test_loss < best_loss:
            best_loss = avg_test_loss
            early_stop_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            early_stop_counter += 1
            if early_stop_counter >= 10:
                print("Early stopping triggered")
                break
    
    return history

# تابع اصلی
def main():
    # بارگذاری و پیش‌پردازش داده
    df = load_and_preprocess('XAUUSD-I_M30.txt')
    
    # آماده‌سازی داده
    sequences, targets, scaler = prepare_data(df)
    
    # تقسیم داده
    train_size = int(0.8 * len(sequences))
    train_seq, test_seq = sequences[:train_size], sequences[train_size:]
    train_tgt, test_tgt = targets[:train_size], targets[train_size:]
    
    # ایجاد DataLoader
    train_loader = DataLoader(
        PriceDataset(train_seq, train_tgt),
        batch_size=BATCH_SIZE,
        shuffle=True
    )
    test_loader = DataLoader(
        PriceDataset(test_seq, test_tgt),
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    
    # ایجاد و آموزش مدل
    model = EnhancedLSTM(input_size=train_seq.shape[2])
    history = train_model(model, train_loader, test_loader)
    
    # ذخیره مدل نهایی
    torch.save({
        'model_state': model.state_dict(),
        'scaler': scaler,
        'config': {
            'input_size': train_seq.shape[2],
            'seq_length': SEQ_LENGTH
        }
    }, 'final_model.pth')
    
    # رسم نمودارها
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['test_loss'], label='Test Loss')
    plt.legend()
    plt.title('Training Loss')
    
    plt.subplot(1, 2, 2)
    plt.plot(history['r2'], label='R² Score')
    plt.legend()
    plt.title('Model Performance')
    plt.show()

if __name__ == "__main__":
    main()

TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'